In [2]:
import os
import random

import cv2
import keras.utils
import numpy as np
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten,Concatenate
from keras.models import Model

keras.utils.set_random_seed(42)
np.random.seed(42)
random.seed(42)



inp = Input(shape=(320, 240, 1))
height = Input(shape=(1,))

n_filters = 512
max_p = inp

for _ in range(3):
    conv = Conv2D(filters=n_filters, kernel_size=5, activation='relu')(max_p)
    max_p = MaxPooling2D(pool_size=(2, 2))(conv)
    
    n_filters //= 2

conv = Conv2D(filters=n_filters, kernel_size=3, activation='relu')(max_p)
max_p = MaxPooling2D(pool_size=(2, 2))(conv)

flatten = Flatten()(max_p)
concat = Concatenate()([flatten, height])
dense = Dense(128, activation='relu')(flatten)

chest_out = Dense(1, activation='linear', name='chest_out')(dense)
waist_out = Dense(1, activation='linear', name='waist_out')(dense)
pelvis_out = Dense(1, activation='linear', name='pelvis_out')(dense)
bicep_out = Dense(1, activation='linear', name='bicep_out')(dense)
thigh_out = Dense(1, activation='linear', name='thigh_out')(dense)
shoulder_to_wrist_out = Dense(1, activation='linear', name='shoulder_to_wrist_out')(dense)
leg_out = Dense(1, activation='linear', name='leg_out')(dense)
calf_out = Dense(1, activation='linear', name='calf_out')(dense)
wrist_out = Dense(1, activation='linear', name='wrist_out')(dense)
shoulder_to_shoulder_out = Dense(1, activation='linear', name='shoulder_to_shoulder_out')(dense)




model = Model(
    inputs=[inp, height],
    outputs=[
        chest_out,
        waist_out,
        pelvis_out,
        bicep_out,
        thigh_out,
        shoulder_to_wrist_out,
        leg_out,
        calf_out,
        wrist_out,
        shoulder_to_shoulder_out,
    ],
    name='conv_bodies'
)


###TRAINING VARIABLES

sample_count_surreact = 79999
sample_count_bodym = 6134
epoch_count = 100
batch_size = 5
validation_count = 20000
initial_learning_rate=.001
decay_steps=8888
decay_rate=0.97
model_name="BoMNHeightSR"

folder_path = f"{model_name} - {sample_count_surreact+sample_count_bodym}s{epoch_count}e{validation_count}v {"" if initial_learning_rate == 0.001 else initial_learning_rate + "ilr"}{decay_steps}s{decay_rate}d"

try:
    os.mkdir(os.path.join("./logs/" + folder_path))
except OSError as error:
    print(error)
    

#model.save_weights("default.weights.h5")
# 
# model.summary()
# 
# from keras.utils import plot_model
# 
# plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True, show_layer_activations=True)

2024-05-26 17:05:36.240276: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-26 17:05:37.373588: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[Errno 17] File exists: './logs/BoMNHeightSR - 86133s100e20000v 8888s0.97d'


In [9]:
from DatasetLoaderClass import DatasetLoader
quickTrain = {
                'image_location': ["Export/Surreact-APose/train/", "Export/BodyM/train/"], 
                'data_location': ["Export/Surreact-APose/train/measurements/", "Export/BodyM/train/measurements/"], 
                'samples': [sample_count_surreact, sample_count_bodym], 
                'dataset': ["Surreact", "BodyM"],
                'batch_size': batch_size, 
                'seed':  69,#np.random.randint(0, 10000), 
                'shuffle': True,
                'height': True}

quickValidate = {
                'image_location': ["Export/Surreact-APose/test/"],
                'data_location': ["Export/Surreact-APose/test/measurements/"],
                'samples': [validation_count],
                'batch_size': batch_size,
                'dataset': ["Surreact"],
                'seed': 69, #np.random.randint(0, 10000),
                'height':True
                }

train_generator = DatasetLoader(**quickTrain)
validation_generator = DatasetLoader(**quickValidate)

In [4]:
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.losses import MeanSquaredError

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True)

stop_no_improvement = keras.callbacks.EarlyStopping(monitor="loss", patience=3)

model.compile(
    optimizer=Adam(learning_rate=lr_schedule),
    loss={
        'chest_out': MeanSquaredError(),
        'waist_out': MeanSquaredError(),
        'pelvis_out': MeanSquaredError(),
        'bicep_out': MeanSquaredError(),
        'thigh_out': MeanSquaredError(),
        'shoulder_to_wrist_out': MeanSquaredError(),
        'leg_out': MeanSquaredError(),
        'calf_out': MeanSquaredError(),
        'wrist_out': MeanSquaredError(),
        'shoulder_to_shoulder_out': MeanSquaredError(),
    },
    metrics={
        'chest_out': ['mae'],
        'waist_out': ['mae'],
        'pelvis_out': ['mae'],
        'bicep_out': ['mae'],
        'thigh_out': ['mae'],
        'shoulder_to_wrist_out': ['mae'],
        'leg_out': ['mae'],
        'calf_out': ['mae'],
        'wrist_out': ['mae'],
        'shoulder_to_shoulder_out': ['mae'],

    },
)

checkpoint_filepath = f'./models/{model_name}.checkpoint.model.keras'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='loss',
    mode='min',
    save_best_only=True)

model.fit(
    x=train_generator,
    validation_data=validation_generator,
    callbacks=[

        TensorBoard(write_graph=True, log_dir="./logs/" + folder_path),
        model_checkpoint_callback,
        stop_no_improvement
    ],
    batch_size=batch_size,
    epochs=epoch_count,
    verbose=1
)

model.save(f"./models/fullSR/{model_name}.keras") 

2024-05-23 16:03:02.698303: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-23 16:03:02.701801: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-23 16:03:02.701992: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Epoch 1/100
   26/17226 ━━━━━━━━━━━━━━━━━━━━ 1:30:43 317ms/step - bicep_out_mae: 51.6205 - calf_out_mae: 45.1255 - chest_out_mae: 163.5504 - leg_out_mae: 87.3420 - loss: 262274.6250 - pelvis_out_mae: 110.1568 - shoulder_to_shoulder_out_mae: 49.3868 - shoulder_to_wrist_out_mae: 68.6066 - thigh_out_mae: 29.8370 - waist_out_mae: 107.5887 - wrist_out_mae: 36.8808

Process Keras_worker_ForkPoolWorker-4:
Process Keras_worker_ForkPoolWorker-1:
Process Keras_worker_ForkPoolWorker-2:
Process Keras_worker_ForkPoolWorker-3:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.12/multiprocessing/process.py", line 108, in run
    se

KeyboardInterrupt: 